<a href="https://colab.research.google.com/github/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes/blob/main/AML_AnomalySegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Real-time Anomaly Segmentation for Road Scenes**

Existing deep neural networks, when deployed in open-world settings, perform poorly on unknown, anomaly, out-of-distribution (OoD) objects that were not present during the training. The goal of this project is to build tiny anomaly segmentation models to segment anomaly patterns. Models must be able to fit in small devices, which represents a realistic memory constraint for an edge application.

## Preparation

In [1]:
!rm -r sample_data/

In [2]:
# download required packages and import useful modules
!pip3 install --quiet numpy
!pip3 install --quiet Pillow

!pip3 install --quiet gdown
!pip3 install --quiet torchvision
!pip3 install --quiet ood_metrics
!pip3 install --quiet cityscapesscripts

!pip3 install --quiet matplotlib
!pip3 install --quiet visdom

import os, sys, subprocess, torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

The following function is implemented to download the *Cityscapes* dataset in two different ways: via Google Drive (using `gdown`) or directly from the Cityscapes official website (using `csDownload`). Although the first option is preferable as it is definitely faster, direct download from the website is provided as an alternative. `gdown` may in fact raise the error *Failed to retrieve the file url* if the file we are attempting to download is exceptionally large (*e.g.* 11G), there are numerous users simultaneously trying to download it programmatically or we download it many times in a limited time. Regardless of the method used, use the conversor (available [here](https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/preparation/createTrainIdLabelImgs.py)) to generate labelTrainIds from labelIds.

In [3]:
def download_cityscapes():

    if not os.path.isdir('/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes'):
        print("Attempting to download cityscapes dataset using gdown...")

        try:
            # If check is true, and the process exits with a non-zero exit code, a CalledProcessError exception will be raised.
            subprocess.run(["gdown", "https://drive.google.com/uc?id=11gSQ9UcLCnIqmY7srG2S6EVwV3paOMEq"], check=True)
            print("Dataset downloaded successfully using gdown. Unzipping...")
            subprocess.run(["unzip", "-q", "cityscapes.zip"], check=True)
            # Use the conversor to generate labelTrainIds from labelIds
            print("Generating trainIds from labelIds...")
            !CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

        except subprocess.CalledProcessError as e:
            print("gdown failed. Attempting to download cityscapes dataset from the official website...")
            try:
              # Cityscapes credentials: (agnesere, FCSBwcVMi-u9-Zn)
              !csDownload leftImg8bit_trainvaltest.zip
              !csDownload gtFine_trainvaltest.zip

              print("Dataset downloaded successfully from the official website. Unzipping...")
              !unzip -q 'leftImg8bit_trainvaltest.zip' -d 'cityscapes'
              !unzip -o -q 'gtFine_trainvaltest.zip' -d 'cityscapes'

              print("Generating trainIds from labelIds...")
              !CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

              print("Cityscapes dataset ready")

            except Exception as e2:
                print("Failed to download the dataset using both methods.")

Download and unzip the validation dataset (*FS_LostFound_full*, *RoadAnomaly*, *RoadAnomaly21*, *RoadObsticle21*, *fs_static*), clone or update the GitHub repository (*Real-Time-Anomaly-Segmentation-for-Road-Scenes*) and download the *Cityscapes* dataset.

In [4]:
# download and unzip validation dataset
if not os.path.isdir('/content/validation_dataset'):
  !gdown 'https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta'
  !unzip -q 'Validation_Dataset.zip'
  !mkdir validation_dataset && cp -pR Validation_Dataset/* validation_dataset/ && rm -R Validation_Dataset/
  !rm 'Validation_Dataset.zip'

# clone the github repo and pull command
if not os.path.isdir('content/Real-Time-Anomaly-Segmentation-for-Road-Scenes'):
  !git clone https://github.com/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes.git
else: # if folder already present
  !git pull

%cd Real-Time-Anomaly-Segmentation-for-Road-Scenes

Downloading...
From (original): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
From (redirected): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta&confirm=t&uuid=15f8a59a-9b03-472b-ba87-a85454bb2606
To: /content/Validation_Dataset.zip
100% 329M/329M [00:04<00:00, 81.3MB/s]
Cloning into 'Real-Time-Anomaly-Segmentation-for-Road-Scenes'...
remote: Enumerating objects: 857, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 857 (delta 59), reused 60 (delta 47), pack-reused 780 (from 1)
Receiving objects: 100% (857/857), 82.01 MiB | 20.66 MiB/s, done.
Resolving deltas: 100% (501/501), done.
/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes


In [5]:
# download cityscapes dataset
download_cityscapes()

Attempting to download cityscapes dataset using gdown...
gdown failed. Attempting to download cityscapes dataset from the official website...
Cityscapes username or email address: agnesere
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: N
Download progress:  98% 10.8G/11.0G [08:18<00:09, 23.3MB/s]
Cityscapes username or email address: agnesere
Cityscapes password: 
Store credentials unencrypted in '/root/.local/share/cityscapesscripts/credentials.json' [y/N]: N
Download progress: 100% 241M/241M [00:03<00:00, 72.9MB/s]
Dataset downloaded successfully from the official website. Unzipping...
Generating trainIds from labelIds...
Processing 5000 annotation files
Progress: 100.0 % Cityscapes dataset ready


## Evaluation

### Step 2A
### Compute AuPRC & FPR95TPR

In [6]:
%cd eval

/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval


In [ ]:
# datasets = os.listdir("../../validation_dataset")
datasets = {
    "SMIYC RA-21": "RoadAnomaly21",
    "SMIYC RO-21": "RoadObsticle21",
    "FS L&F": "FS_LostFound_full",
    " FS Static": "fs_static",
    "Road Anomaly": "RoadAnomaly"
    }

Perform various anomaly inferences using the pre-trained **ErfNet** model and anomaly segmentation test dataset provided. Different techniques are used (MSP, MaxLogit and MaxEntropy).

In [ ]:
methods = ["MSP", "MaxLogit", "MaxEntropy"]

for dataset, folder in datasets.items():
  print(f"Dataset {dataset}")

  for method in methods:
    print(f" - {method:<10} ", end = "")
    input_path = f"../../validation_dataset/{folder}/images/*.*"
    plot_dir_path = f"../plots/baselines/{folder}_{method}"
    if torch.cuda.is_available():
      !python evalAnomaly.py --input={input_path} --method={method} --plotdir={plot_dir_path}
    else:
      !python evalAnomaly.py --input={input_path} --method={method} --plotdir={plot_dir_path} --cpu

  print("=" * 55, end = "\n")

Dataset SMIYC RA-21
 - MSP        | AUPRC score: 29.100 | FPR@TPR95: 62.511
 - MaxLogit   | AUPRC score: 38.320 | FPR@TPR95: 59.337
 - MaxEntropy | AUPRC score: 31.005 | FPR@TPR95: 62.593
Dataset SMIYC RO-21
 - MSP        | AUPRC score: 2.712 | FPR@TPR95: 64.974
 - MaxLogit   | AUPRC score: 4.627 | FPR@TPR95: 48.443
 - MaxEntropy | AUPRC score: 3.052 | FPR@TPR95: 65.600
Dataset FS L&F
 - MSP        | AUPRC score: 1.748 | FPR@TPR95: 50.763
 - MaxLogit   | AUPRC score: 3.301 | FPR@TPR95: 45.495
 - MaxEntropy | AUPRC score: 2.582 | FPR@TPR95: 50.368
Dataset  FS Static
 - MSP        | AUPRC score: 7.470 | FPR@TPR95: 41.823
 - MaxLogit   | AUPRC score: 9.499 | FPR@TPR95: 40.300
 - MaxEntropy | AUPRC score: 8.826 | FPR@TPR95: 41.523
Dataset Road Anomaly
 - MSP        | AUPRC score: 12.426 | FPR@TPR95: 82.492
 - MaxLogit   | AUPRC score: 15.582 | FPR@TPR95: 73.248
 - MaxEntropy | AUPRC score: 12.678 | FPR@TPR95: 82.632


If you want to save the baselines folder in your local machine, create a ZIP file with the following command and then download it.

In [ ]:
 # !zip -r baselines.zip baselines/

### Compute mIoU

In [ ]:
if torch.cuda.is_available():
  !python eval_iou.py --loadDir ../trained_models/ --datadir /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes
else:
  !python eval_iou.py --loadDir ../trained_models/ --datadir /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes --cpu

Loading model: ../trained_models/erfnet.py
Loading weights: ../trained_models/erfnet_pretrained.pth
Model and weights LOADED successfully
/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes/leftImg8bit/val /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/cityscapes/gtFine/val
---------------------------------------
Took  78.16403198242188 seconds
Per-Class IoU:
97.62 Road
81.37 sidewalk
90.77 building
49.43 wall
54.93 fence
60.81 pole
62.60 traffic light
72.31 traffic sign
91.35 vegetation
60.96 terrain
93.38 sky
76.11 person
53.45 rider
92.91 car
72.78 truck
78.87 bus
63.86 train
46.40 motorcycle
71.89 bicycle
MEAN IoU:  72.20 %


### Step 2B - Compute AuPRC & FPR95TPR with temperature scaling

In [ ]:
temperatures = [0.5, 0.75, 1.0, 1.1, 1.2, 1.5, 2.0, 5.0, 10.0]

for dataset, folder in datasets.items():
  print(f"Dataset {dataset}")

  for temperature in temperatures:
    print(f" - {temperature:<10} ", end = "")
    input_path = f"../../validation_dataset/{folder}/images/*.*"
    # plot_dir_path = f"../plots/temperature/{folder}_MSP"
    if torch.cuda.is_available():
      !python evalAnomaly.py --input={input_path} --method="MSP" --temperature={temperature}
    else:
      !python evalAnomaly.py --input={input_path} --method="MSP" --temperature={temperature} --cpu

  print("=" * 55, end = "\n")

Dataset SMIYC RA-21
 - 0.5        | AUPRC score: 27.061 | FPR@TPR95: 62.731
 - 0.75       | AUPRC score: 28.156 | FPR@TPR95: 62.479
 - 1.0        | AUPRC score: 29.100 | FPR@TPR95: 62.511
 - 1.1        | AUPRC score: 29.410 | FPR@TPR95: 62.590
 - 1.2        | AUPRC score: 29.678 | FPR@TPR95: 62.724
 - 1.5        | AUPRC score: 30.258 | FPR@TPR95: 63.318
 - 2.0        | AUPRC score: 30.679 | FPR@TPR95: 64.721
 - 5.0        | AUPRC score: 30.196 | FPR@TPR95: 71.594
 - 10.0       | AUPRC score: 29.526 | FPR@TPR95: 75.757
Dataset SMIYC RO-21
 - 0.5        | AUPRC score: 2.420 | FPR@TPR95: 63.225
 - 0.75       | AUPRC score: 2.567 | FPR@TPR95: 64.053
 - 1.0        | AUPRC score: 2.712 | FPR@TPR95: 64.974
 - 1.1        | AUPRC score: 2.766 | FPR@TPR95: 65.524
 - 1.2        | AUPRC score: 2.816 | FPR@TPR95: 66.033
 - 1.5        | AUPRC score: 2.937 | FPR@TPR95: 67.928
 - 2.0        | AUPRC score: 3.026 | FPR@TPR95: 71.459
 - 5.0        | AUPRC score: 2.841 | FPR@TPR95: 83.111
 - 10.0       | 

### Step 3 - Train models with void classifier

In [7]:
models = ["erfnet", "enet", "bisenet"]
savedirs = ["erfnet_training_void", "enet_training_void", "bisenet_training_void"]
pretrained_weights = ["erfnet_pretrained.pth", "enet_pretrained.pth", "bisenetv1_pretrained.pth"]

base_dir = "../train"
data_dir = "../cityscapes"

In [ ]:
!cd {base_dir} && python -W ignore main_v2.py --savedir "bisenet_training_void_1" --datadir {data_dir} --model "bisenet" --cuda --num-epochs=20 --epochs-save=1 --batch-size=6
#lanciare come python main_v2.py --savedir "nomerete_trainig_void" --datadir {data_dir} --model "nomemodello" --cuda --num-epochs=20 --epochs-save=1 --batch-size=6
#rilanciare con parametro per pesi pretrainati

### BiSeNet first training

In [8]:
!cd {base_dir} && python -W ignore main_v2.py --savedir "bisenet_training_void_1" --datadir {data_dir} --model "bisenet" --cuda --num-epochs=20 --epochs-save=1 --batch-size=6


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100% 44.7M/44.7M [00:00<00:00, 359MB/s]
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.025
loss: 9.474 (epoch: 1, step: 0) // Avg time/img: 0.3946 s
loss: 5.458 (epoch: 1, step: 50) // Avg time/img: 0.0492 s
loss: 5.002 (epoch: 1, step: 100) // Avg time/img: 0.0461 s
loss: 4.793 (epoch: 1, step: 150) // Avg time/img: 0.0447 s
loss: 4.705 (epoch: 1, step: 200) // Avg time/img: 0.0440 s
loss: 4.626 (epoch: 1, step: 250) // Avg time/img: 0.0438 s
loss: 4.583 (epoch: 1, step: 300) // Avg time/img: 0.0435 s
loss: 4.533 (epoch: 1, step: 350) // Avg time/img: 0.0433 s
loss: 4.506 (epoch: 1, step: 400) // Avg time/img: 0.0431 s
loss: 4.465 (epoch: 1, step: 450) // Avg time/img: 0.0431 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 3.579 (epoch: 1, step: 0) // Avg time/im

### ERFNet Fine-tuning

In [ ]:
!cd {base_dir} && python -W ignore main_v2.py --savedir "erfnet_training_void" --datadir {data_dir} --model "erfnet" --cuda --num-epochs=20 --epochs-save=1 --FineTune --decoder --loadWeights="erfnet_pretrained.pth"

Import Model erfnet with weights erfnet_pretrained.pth to FineTune
========== TRAINING ===========
../cityscapes/leftImg8bit/train
../cityscapes/leftImg8bit/val
ERFNet criterion: <class 'utils.losses.ce_loss.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  5e-05
loss: 0.3277 (epoch: 1, step: 0) // Avg time/img: 0.1345 s
loss: 0.3739 (epoch: 1, step: 50) // Avg time/img: 0.0380 s
loss: 0.3833 (epoch: 1, step: 100) // Avg time/img: 0.0370 s
loss: 0.3838 (epoch: 1, step: 150) // Avg time/img: 0.0369 s
loss: 0.3784 (epoch: 1, step: 200) // Avg time/img: 0.0372 s
loss: 0.3808 (epoch: 1, step: 250) // Avg time/img: 0.0374 s
loss: 0.3803 (epoch: 1, step: 300) // Avg time/img: 0.0374 s
loss: 0.3776 (epoch: 1, step: 350) // Avg time/img: 0.0374 s
loss: 0.3748 (epoch: 1, step: 400) // Avg time/img: 0.0374 s
loss: 0.3732 (epoch: 1, step: 450) // Avg time/img: 0.0375 s
----- VALIDATING - EPOCH 1 -----
VAL loss: 0.4784 (epoch: 1, step: 0) // Avg time/img: 0.0324 s
VAL loss: 0.576

In [9]:
print(os.getcwd())

/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval


In [10]:
%cd ../save

/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save


In [11]:
!zip -r bisenet_training_void.zip bisenet_training_void_1/

  adding: bisenet_training_void_1/ (stored 0%)
  adding: bisenet_training_void_1/model-020.pth (deflated 7%)
  adding: bisenet_training_void_1/model-005.pth (deflated 7%)
  adding: bisenet_training_void_1/model-002.pth (deflated 7%)
  adding: bisenet_training_void_1/model-014.pth (deflated 7%)
  adding: bisenet_training_void_1/model-004.pth (deflated 7%)
  adding: bisenet_training_void_1/opts.txt (deflated 38%)
  adding: bisenet_training_void_1/model-008.pth (deflated 7%)
  adding: bisenet_training_void_1/model_best.pth.tar (deflated 7%)
  adding: bisenet_training_void_1/model-003.pth (deflated 7%)
  adding: bisenet_training_void_1/model-012.pth (deflated 7%)
  adding: bisenet_training_void_1/model.txt (deflated 91%)
  adding: bisenet_training_void_1/model-011.pth (deflated 7%)
  adding: bisenet_training_void_1/checkpoint.pth.tar (deflated 7%)
  adding: bisenet_training_void_1/automated_log.txt (deflated 59%)
  adding: bisenet_training_void_1/model-017.pth (deflated 7%)
  adding: bisen

# REVISE

**Evaluation**

In [ ]:
import torch

no_execute = False
just_once = False

for model in models:
  print("----------------------------")
  for dataset_dir in datasets_list:

    if no_execute:
      break

    load_dir = f'content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{net}_training_void'
    weights = f'/model_best.pth'
    format_file = os.listdir(f'/content/validation_dataset/{dataset_dir}/images')[0].split(".")[1]
    input =f'/content/validation_dataset/{dataset_dir}/images/\*.{format_file}'
    print(f"\nDataset: {dataset_dir} net: {net}")

    if torch.cuda.is_available():
      !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} | tail -n 2
    else:
      !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --cpu | tail -n 2

    if just_once:
      no_execute = True
      just_once = False


**mIoU Void Classification**

In [ ]:
import torch
no_execute = False
just_once = False

for model in models:
  print("----------------------------")

  if no_execute:
      break
  print(f"-----------{model}-------------")
  loadDir = f'content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{model}_training_void'
  weights = f'/model_best.pth'
  if torch.cuda.is_available():
    !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/eval_iou.py --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/ --model {model} | tail -n 25
  else:
    !python  content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/eval_iou.py  --loadDir {loadDir} --loadWeights {weights} --datadir /content/cityscapes/  --model {model}  --cpu | tail -n 25


  if just_once:
    no_execute = True
    just_once = False

### Step 4 - Analyze the Effect of Training Loss function

Analyze the effect of the training model along with losses that are specifically made for anomaly detection.

**Losses**

**Fine-tuning**

In [ ]:
# Fine tune ERFNET with different losses
"""
Training:
1. Focal loss
2. LogitNorm+CrossEntropy loss
3. IsoMaxPlus+CrossEntropy loss
4. LogitNorm+Focal loss
5. IsoMaxPlus+Focal loss
"""
titles = ["Focal", "LogitNorm+CrossEntropy", "IsoMaxPlus+CrossEntropy", "LogitNorm+Focal", "IsoMaxPlus+Focal"]
losses = ["Focal", "CrossEntropy", "CrossEntropy", "Focal", "Focal"]
models = ["erfnet", "erfnet", "erfnet_isomaxplus", "erfnet", "erfnet_isomaxplus"]
savedirs = ["erfnet_training_focal_loss", "erfnet_training_logitnorm_cross_entropy_loss", "erfnet_training_isomaxplus_cross_entropy_loss", "erfnet_training_logitnorm_focal_loss", "erfnet_training_isomaxplus_focal_loss"]
logit_normalization_flags = [False, True, False, True, False]
epochs = 20

# Base directory of the project
base_dir = "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/train"
# Dataset directory
data_dir = "/content/cityscapes"
pretrained_weights = "erfnet_pretrained.pth"

# Loop to execute fine-tuning
for title, loss, model, savedir, logit_normalization_flag in zip(titles, losses, models, savedirs, logit_normalization_flags):
    print(f"\n\n----- Fine-tuning with {title} loss -----")
    !cd {base_dir} && python -W ignore main.py --savedir {savedir} --loss {loss} --logit_normalization {logit_normalization_flag} --datadir {data_dir} --model {model} --cuda --num-epochs=20 --epochs-save=1 --FineTune --decoder --loadWeights={pretrained_weights}
    print(f"Model saved in /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{savedir}")
    # zip folder
    !zip -r save_{savedir}.zip /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{savedir}

**Extension Evaluation**

In [ ]:
import torch

no_execute = False
just_once = False

losses = ["CrossEntropy", "Focal", "LogitNorm+CrossEntropy", "IsoMaxPlus+CrossEntropy", "LogitNorm+FocalLoss", "IsoMaxPlus+FocalLoss"]
models = ["erfnet", "erfnet", "erfnet", "erfnet_isomaxplus", "erfnet", "erfnet_isomaxplus"]
load_dirs = ["/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/trained_models/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_focal_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_logitnorm_cross_entropy_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_isomaxplus_cross_entropy_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_logitnorm_focal_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_isomaxplus_focal_loss/"]
weights = ["erfnet_pretrained.pth", "model_best.pth", "model_best.pth", "model_best.pth", "model_best.pth", "model_best.pth"]

for loss, model, load_dir, weight in zip(losses, models, load_dirs, weights):
  print(f"------ Evaluating loss: {loss} ------\n")
  for dataset_dir in ['RoadAnomaly21', 'RoadObsticle21', 'FS_LostFound_full', 'fs_static', 'RoadAnomaly']:
    for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:

      if no_execute:
        break

      format_file = os.listdir(f'/content/Validation_Dataset/{dataset_dir}/images')[0].split(".")[1]
      input =f'/content/Validation_Dataset/{dataset_dir}/images/\*.{format_file}'

      print(f"\nDataset: {dataset_dir} method: {method} loss: {loss}")

      if torch.cuda.is_available():
        !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --model {model} --loadDir {load_dir} --loadWeights {weight} | tail -n 2
      else:
        !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method}  --model {model} --loadDir {load_dir} --loadWeights {weight} --cpu | tail -n 2

      print("----------------------------")
      if just_once:
        no_execute = True
        just_once = False
    print("----------------------------\n\n")

##Plot

In [ ]:
import torch

# Example image to color
# Nice images: RoadAnomaly/images/28, RoadAnomaly/images/58
input = '/content/Validation_Dataset/RoadAnomaly/images/58.jpg'

### Baseline models ###
for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:
  print(f"Method: {method}")
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline/{method}'

  if torch.cuda.is_available():
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --save-colored {save_image_path}  | tail -n 2
  else:
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method} --save-colored {save_image_path} --cpu | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/ccontent/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/baseline_visualization.png"

### Temperature scaling ###
for t in [0.5, 0.75, 1.1]:
  print(f"Method: MSP, Temperature: {t}")
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/temperature/t={t}'

  if torch.cuda.is_available():
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method 'MSP' --temperature {t} --save-colored {save_image_path} | tail -n 2
  else:
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method 'MSP' --cpu --temperature {t} --save-colored {save_image_path} | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/temperature" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/temperature_visualization.png"

### Finetuned models with void ###
for net in ["erfnet", "enet", "bisenet"]:
  save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/void/{net}'
  load_dir = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/{net}_training_void'
  weights = f'/model_best.pth'
  print(f"Finetuned network: {net}")
  if torch.cuda.is_available():
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --save-colored {save_image_path} | tail -n 2
  else:
    !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --void --model {net} --loadDir {load_dir} --loadWeights {weights} --cpu --save-colored {save_image_path} | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/void" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/void_visualization.png"

### Losses ###
losses = ["CrossEntropy", "Focal", "LogitNorm", "IsoMaxPlus"]
models = ["erfnet", "erfnet", "erfnet", "erfnet_isomaxplus"]
load_dirs = ["/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/trained_models/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_focal_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_logitnorm_loss/", "/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/save/erfnet_training_isomaxplus_loss/"]
weights = ["erfnet_pretrained.pth", "model_best.pth", "model_best.pth", "model_best.pth"]
for loss, model, load_dir, weight in zip(losses, models, load_dirs, weights):
  for method in ["MSP", "MaxLogit", "MaxEntropy", "Mahalanobis"]:
    save_image_path = f'/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/{loss}/{method}'
    print(f"Method: {method}, loss: {loss}")
    if torch.cuda.is_available():
      !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method  {method} --model {model} --loadDir {load_dir} --loadWeights {weight} --save-colored {save_image_path} | tail -n 2
    else:
      !python  /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/evalAnomaly.py --input {input} --method {method}  --model {model} --loadDir {load_dir} --loadWeights {weight} --save-colored {save_image_path} --cpu | tail -n 2

!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/CrossEntropy" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_CrossEntropy_visualization.png"
!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/Focal" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_Focal_visualization.png"
!python /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/LogitNorm" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_LogitNorm_visualization.png"
!python /ccontent/Real-Time-Anomaly-Segmentation-for-Road-Scenes/eval/visualization.py --name_dir="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses/IsoMaxPlus" --name_output="/content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization/losses_IsoMaxPlus_visualization.png"

# Zip the images
!zip -r colored_anomalies.zip /content/Real-Time-Anomaly-Segmentation-for-Road-Scenes/visualization